<a href="https://colab.research.google.com/github/Johanhms/Learning_Python/blob/main/Practica_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install pyspark==3.5.0

# Importar las bibliotecas necesarias
import os
from pyspark.sql import SparkSession

# Establecer las variables de entorno
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

In [25]:
import seaborn as sns
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("LeerCSV") \
    .getOrCreate()


In [26]:
df_products = spark.read.csv('/content/Products.csv', header = True, inferSchema = True)
df_products.show(5)

+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               NULL|        59.98|http://images.acm...|
|         2|                  2|Under Armour Men'...|               NULL|       129.99|http://images.acm...|
|         3|                  2|Under Armour Men'...|               NULL|        89.99|http://images.acm...|
|         4|                  2|Under Armour Men'...|               NULL|        89.99|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|               NULL|       199.99|http://images.acm...|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
only showing top 5 

In [27]:
df_categories = spark.read.csv('/content/categories.csv', header = True, inferSchema = True)
df_categories.show(5)

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
|          4|                     2|         Basketball|
|          5|                     2|           Lacrosse|
+-----------+----------------------+-------------------+
only showing top 5 rows



In [28]:
df_customer = spark.read.csv('/content/customers.csv', header = True, inferSchema = True)
df_customer.show(5)

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|             725|
|          4|          Mary|         Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|            CA|          

In [29]:
df_departments = spark.read.csv('/content/departments.csv', header = True, inferSchema = True)
df_departments.show(5)

+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|            2|        Fitness|
|            3|       Footwear|
|            4|        Apparel|
|            5|           Golf|
|            6|       Outdoors|
+-------------+---------------+
only showing top 5 rows



In [30]:
df_order_items = spark.read.csv('/content/order_items.csv', header = True, inferSchema = True)
df_order_items.show(5)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            5|                  4|                  897|                  2|              49.98|                   24.99|
+-------------+-

In [31]:
df_orders = spark.read.csv('/content/orders.csv', header = True, inferSchema = True)
df_orders.show(5)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
+--------+-------------------+-----------------+---------------+
only showing top 5 rows



In [32]:
df_join = df_orders.join(df_customer, df_orders.order_customer_id == df_customer.customer_id)
df_join.show(5)

+--------+-------------------+-----------------+---------------+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|order_id|         order_date|order_customer_id|   order_status|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+--------+-------------------+-----------------+---------------+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|      11599|          Mary|        Malone|     XXXXXXXXX|        XXXXXXXXX|8708 Indian Horse...|      Hickory|            NC|           28601|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|        256|         David|     Rodriguez|     XXXXXXXXX|        XXXXXXXXX|7605 Tawny Horse ...|      Chicag

In [34]:
# Realizar múltiples joins en una sola sentencia
df_resultados = df_order_items.join(
    df_orders,
    df_order_items.order_item_order_id == df_orders.order_id,
    "left"
).join(
    df_customer,
    df_orders.order_customer_id == df_customer.customer_id,
    "left"
).join(
    df_products,
    df_order_items.order_item_product_id == df_products.product_id,
    "left"
).join(
    df_categories,
    df_products.product_category_id == df_categories.category_id,
    "left"
).join(
    df_departments,
    df_categories.category_department_id == df_departments.department_id,
    "left"
)

# Seleccionar solo las columnas de interés para un resultado más limpio
df_final = df_resultados.select(
    "order_id",
    "order_status",
    "customer_fname",
    "customer_lname",
    "product_name",
    "product_price",
    "order_item_quantity",
    "order_item_subtotal",
    "department_name"
)

# Mostrar el resultado final
print("Resultado de la unión de múltiples tablas:")
df_final.show(truncate=False)

Resultado de la unión de múltiples tablas:
+--------+---------------+--------------+--------------+---------------------------------------------+-------------+-------------------+-------------------+---------------+
|order_id|order_status   |customer_fname|customer_lname|product_name                                 |product_price|order_item_quantity|order_item_subtotal|department_name|
+--------+---------------+--------------+--------------+---------------------------------------------+-------------+-------------------+-------------------+---------------+
|1       |CLOSED         |Mary          |Malone        |Diamondback Women's Serene Classic Comfort Bi|299.98       |1                  |299.98             |Fan Shop       |
|2       |PENDING_PAYMENT|David         |Rodriguez     |Pelican Sunstream 100 Kayak                  |199.99       |1                  |199.99             |Fan Shop       |
|2       |PENDING_PAYMENT|David         |Rodriguez     |Nike Men's Dri-FIT Victory Golf Polo

In [39]:
from pyspark.sql.functions import *

# 1. Agrupar por cliente y categoría, y sumar los subtotales de los pedidos
df_ranking = df_resultados.groupBy("customer_fname", "customer_lname", "category_name") \
    .agg(round(sum("order_item_subtotal"),2).alias("total_comprado"))

# 2. Ordenar los resultados para crear el ranking
# Ordenamos de forma descendente (desc) para ver a los clientes con el mayor gasto primero
df_ranking_final = df_ranking.orderBy(desc("total_comprado"))

# 3. Mostrar el ranking final
print("Ranking de clientes por gasto en cada categoría:")
df_ranking_final.show(truncate=False)

Ranking de clientes por gasto en cada categoría:
+--------------+--------------+--------------------+--------------+
|customer_fname|customer_lname|category_name       |total_comprado|
+--------------+--------------+--------------------+--------------+
|Mary          |Smith         |Fishing             |827558.62     |
|Mary          |Smith         |Cleats              |537270.44     |
|Mary          |Smith         |Camping & Hiking    |502466.5      |
|Mary          |Smith         |Cardio Equipment    |439366.5      |
|Mary          |Smith         |Women's Apparel     |380150.0      |
|Mary          |Smith         |Water Sports        |375781.21     |
|Mary          |Smith         |Indoor/Outdoor Games|345411.78     |
|Mary          |Smith         |Men's Footwear      |336544.11     |
|Mary          |Smith         |Shop By Sport       |153042.92     |
|Mary          |Smith         |Electronics         |53326.46      |
|James         |Smith         |Fishing             |26398.68      |

In [45]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# 1. Definir la ventana
# Particionamos por cliente y categoría, y ordenamos por el subtotal del pedido
# Esto crea una "ventana" única para cada cliente en cada categoría.
window_spec = Window.partitionBy("customer_fname", "customer_lname", "category_name") \
    .orderBy(F.desc("order_item_subtotal")) # Use F.desc for ordering within the window

# 2. Aplicar la función de suma acumulada (cumulative sum)
# Calculamos el total acumulado de las compras dentro de cada partición
df_ranking_con_acumulado = df_resultados.withColumn(
    "total_acumulado_por_categoria",
    F.round(F.sum("order_item_subtotal").over(window_spec), 2) # Use F.sum and F.round
)

# 3. Mostrar el resultado
# Seleccionamos las columnas relevant es para ver el efecto de la suma acumulada
df_ranking_con_acumulado.select(
    "customer_fname",
    "customer_lname",
    "category_name",
    "order_item_subtotal",
    "total_acumulado_por_categoria"
).show()

+--------------+--------------+----------------+-------------------+-----------------------------+
|customer_fname|customer_lname|   category_name|order_item_subtotal|total_acumulado_por_categoria|
+--------------+--------------+----------------+-------------------+-----------------------------+
|         Aaron|        Berger|Camping & Hiking|             299.98|                       299.98|
|         Aaron|        Berger|     Electronics|             159.95|                       159.95|
|         Aaron|        Berger|    Water Sports|             199.99|                       399.98|
|         Aaron|        Berger|    Water Sports|             199.99|                       399.98|
|         Aaron|        Berger| Women's Apparel|              100.0|                        100.0|
|         Aaron|         Boyle|  Men's Footwear|             129.99|                       129.99|
|         Aaron|         Boyle|   Shop By Sport|              39.99|                        79.98|
|         